In [10]:
import pandas as pd

df = pd.read_csv('cust_act1.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   administrative            11760 non-null  float64
 1   administrative_duration   11418 non-null  float64
 2   informational             12330 non-null  int64  
 3   informational_duration    11994 non-null  float64
 4   product_related           11751 non-null  float64
 5   product_related_duration  12129 non-null  float64
 6   bounce_rates              12330 non-null  float64
 7   exit_rates                12330 non-null  float64
 8   page_values               12330 non-null  float64
 9   month                     12330 non-null  object 
 10  operating_systems         12319 non-null  object 
 11  browser                   12330 non-null  object 
 12  region                    12330 non-null  object 
 13  traffic_type              12330 non-null  object 
 14  visito

In [ ]:
import plotly.express as px

fig = px.histogram(df, "browser", 
                   facet_col="region", 
                   color="region",
                   title="Counts of browser per region",
                   labels={"browser": "browser"},
                   height=1000, 
                   facet_col_wrap=2, 
                   facet_col_spacing=0.1,
                   )
fig.update_layout(showlegend=False, xaxis={'categoryorder':'total descending'})
fig.update_yaxes(matches=None, showticklabels=True)
# fig.update_xaxes(showticklabels=True)
fig.show()

In [21]:
import numpy as np

def percentnull(df_column):
    print(f"{np.round(100 * df_column.isna().sum()/len(df_column), 2)}% of values in the <{df_column.name}> column are null.")

percentnull(df["product_related"])

4.7% of values in the <product_related> column are null.
